In [105]:
import pandas as pd
import numpy as np
import requests
import json
import openmeteo_requests
import requests_cache
from retry_requests import retry
import csv

In [106]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)
url = "https://archive-api.open-meteo.com/v1/archive"

In [107]:
meal_hours = [[8, 10], [12, 14], [17, 19]]
locations = {"kuzey":[41.0836, 29.0518], "guney":[41.0836, 29.0518], "hisar":[41.0892, 29.0509], "kilyos":[41.2446, 29.0128], "kandilli":[41.0638, 29.0653], "ahisari":[41.0836, 29.0518]}

In [108]:
df = pd.read_csv("data/dataset_910_without_temp.csv")

In [109]:
df

,date,day,campus,count,meal,soup,main_dish,vegetarian,side_dishes,dessert
0,2024-09-02,monday,Anadoluhisari,17,lunch,tavuk suyu corba,rosto kofte,patates oturtma,"misirli pirinc pilavi, eriste","meyve, coban salata, ayran"
1,2024-09-02,monday,Kuzey,268,lunch,tavuk suyu corba,rosto kofte,patates oturtma,"misirli pirinc pilavi, eriste","meyve, coban salata, ayran"
2,2024-09-02,monday,Guney,106,lunch,tavuk suyu corba,rosto kofte,patates oturtma,"misirli pirinc pilavi, eriste","meyve, coban salata, ayran"
3,2024-09-02,monday,Hisar,29,lunch,tavuk suyu corba,rosto kofte,patates oturtma,"misirli pirinc pilavi, eriste","meyve, coban salata, ayran"
4,2024-09-02,monday,Kilyos,20,lunch,tavuk suyu corba,rosto kofte,patates oturtma,"misirli pirinc pilavi, eriste","meyve, coban salata, ayran"
...,...,...,...,...,...,...,...,...,...,...
311,2024-10-30,wednesday,Anadoluhisari,281,breakfast,NaN,menemen,yumurta,"yesil zeytin, siyah zeytin, domates, salatalik...","kasar peynir, tereyag, recel, tahin-pekmez"
312,2024-10-30,wednesday,Kuzey,181,breakfast,NaN,menemen,yumurta,"yesil zeytin, siyah zeytin, domates, salatalik...","kasar peynir, tereyag, recel, tahin-pekmez"
313,2024-10-30,wednesday,Guney,71,breakfast,NaN,menemen,yumurta,"yesil zeytin, siyah zeytin, domates, salatalik...","kasar peynir, tereyag, recel, tahin-pekmez"
314,2024-10-30,wednesday,Kilyos,12,breakfast,NaN,menemen,yumurta,"yesil zeytin, siyah zeytin, domates, salatalik...","kasar peynir, tereyag, recel, tahin-pekmez"


In [110]:
kuzey_params = {
    "latitude": locations["kuzey"][0],
    "longitude": locations["kuzey"][1],
    "start_date": df['date'].min(),
    "end_date": df['date'].max(),
    "hourly": "temperature_2m"
}

guney_params = {
    "latitude": locations["guney"][0],
    "longitude": locations["guney"][1],
    "start_date": df['date'].min(),
    "end_date": df['date'].max(),
    "hourly": "temperature_2m"
}

hisar_params = {
    "latitude": locations["hisar"][0],
    "longitude": locations["hisar"][1],
    "start_date": df['date'].min(),
    "end_date": df['date'].max(),
    "hourly": "temperature_2m"
}

kilyos_params = {
    "latitude": locations["kilyos"][0],
    "longitude": locations["kilyos"][1],
    "start_date": df['date'].min(),
    "end_date": df['date'].max(),
    "hourly": "temperature_2m"
}

kandilli_params = {
    "latitude": locations["kandilli"][0],
    "longitude": locations["kandilli"][1],
    "start_date": df['date'].min(),
    "end_date": df['date'].max(),
    "hourly": "temperature_2m"
}

ahisari_params = {
    "latitude": locations["ahisari"][0],
    "longitude": locations["ahisari"][1],
    "start_date": df['date'].min(),
    "end_date": df['date'].max(),
    "hourly": "temperature_2m"
}

kuzey_response = openmeteo.weather_api(url, params=kuzey_params)
guney_response = openmeteo.weather_api(url, params=guney_params)
hisar_response = openmeteo.weather_api(url, params=hisar_params)
kilyos_response = openmeteo.weather_api(url, params=kilyos_params)
kandilli_response = openmeteo.weather_api(url, params=kandilli_params)
ahisari_response = openmeteo.weather_api(url, params=ahisari_params)

In [111]:
kuzey_weather = kuzey_response[0].Hourly().Variables(0).ValuesAsNumpy().reshape(-1, 24)
guney_weather = guney_response[0].Hourly().Variables(0).ValuesAsNumpy().reshape(-1, 24)
hisar_weather = hisar_response[0].Hourly().Variables(0).ValuesAsNumpy().reshape(-1, 24)
kilyos_weather = kilyos_response[0].Hourly().Variables(0).ValuesAsNumpy().reshape(-1, 24)
kandilli_weather = kandilli_response[0].Hourly().Variables(0).ValuesAsNumpy().reshape(-1, 24)
ahisari_weather = ahisari_response[0].Hourly().Variables(0).ValuesAsNumpy().reshape(-1, 24)

kuzey_breakfast_weather = kuzey_weather[:, meal_hours[0][0]:meal_hours[0][1]].mean(axis=1)
kuzey_lunch_weather = kuzey_weather[:, meal_hours[1][0]:meal_hours[1][1]].mean(axis=1)
kuzey_dinner_weather = kuzey_weather[:, meal_hours[2][0]:meal_hours[2][1]].mean(axis=1)

guney_breakfast_weather = guney_weather[:, meal_hours[0][0]:meal_hours[0][1]].mean(axis=1)
guney_lunch_weather = guney_weather[:, meal_hours[1][0]:meal_hours[1][1]].mean(axis=1)
guney_dinner_weather = guney_weather[:, meal_hours[2][0]:meal_hours[2][1]].mean(axis=1)

hisar_breakfast_weather = hisar_weather[:, meal_hours[0][0]:meal_hours[0][1]].mean(axis=1)
hisar_lunch_weather = hisar_weather[:, meal_hours[1][0]:meal_hours[1][1]].mean(axis=1)
hisar_dinner_weather = hisar_weather[:, meal_hours[2][0]:meal_hours[2][1]].mean(axis=1)

kilyos_breakfast_weather = kilyos_weather[:, meal_hours[0][0]:meal_hours[0][1]].mean(axis=1)
kilyos_lunch_weather = kilyos_weather[:, meal_hours[1][0]:meal_hours[1][1]].mean(axis=1)
kilyos_dinner_weather = kilyos_weather[:, meal_hours[2][0]:meal_hours[2][1]].mean(axis=1)

kandilli_breakfast_weather = kandilli_weather[:, meal_hours[0][0]:meal_hours[0][1]].mean(axis=1)
kandilli_lunch_weather = kandilli_weather[:, meal_hours[1][0]:meal_hours[1][1]].mean(axis=1)
kandilli_dinner_weather = kandilli_weather[:, meal_hours[2][0]:meal_hours[2][1]].mean(axis=1)

ahisari_breakfast_weather = ahisari_weather[:, meal_hours[0][0]:meal_hours[0][1]].mean(axis=1)
ahisari_lunch_weather = ahisari_weather[:, meal_hours[1][0]:meal_hours[1][1]].mean(axis=1)
ahisari_dinner_weather = ahisari_weather[:, meal_hours[2][0]:meal_hours[2][1]].mean(axis=1)

cevrimdisi_weather = (df['campus'] == "cevrimdisi").sum() * [np.nan]
kuzey_response

In [112]:
start = df['date'].min(),
stop = df['date'].max(),

In [113]:
# generate array of dates
dates = pd.date_range(start[0], stop[0], freq='D')

In [114]:
for i, row in df.iterrows():
    date = row['date']
    idx = dates.get_loc(date)
    if row['campus'].strip() == "Kuzey":
        df.at[i, 'campus'] = 'kuzey'
        if row['meal'].strip() == "breakfast":
            df.at[i, 'weather'] = kuzey_breakfast_weather[idx]
        elif row['meal'].strip() == "lunch":
            df.at[i, 'weather'] = kuzey_lunch_weather[idx]
        elif row['meal'].strip() == "dinner":
            df.at[i, 'weather'] = kuzey_dinner_weather[idx]
    elif row['campus'].strip() == "Guney":
        df.at[i, 'campus'] = 'guney'
        if row['meal'].strip() == "breakfast":
            df.at[i, 'weather'] = guney_breakfast_weather[idx]
        elif row['meal'].strip() == "lunch":
            df.at[i, 'weather'] = guney_lunch_weather[idx]
        elif row['meal'].strip() == "dinner":
            df.at[i, 'weather'] = guney_dinner_weather[idx]
    elif row['campus'].strip() == "Hisar":
        df.at[i, 'campus'] = 'hisar'
        if row['meal'].strip() == "breakfast":
            df.at[i, 'weather'] = hisar_breakfast_weather[idx]
        elif row['meal'].strip() == "lunch":
            df.at[i, 'weather'] = hisar_lunch_weather[idx]
        elif row['meal'].strip() == "dinner":
            df.at[idx, 'weather'] = hisar_dinner_weather[idx]
    elif row['campus'].strip() == "Kilyos":
        df.at[i, 'campus'] = 'kilyos'
        if row['meal'].strip() == "breakfast":
            df.at[i, 'weather'] = kilyos_breakfast_weather[idx]
        elif row['meal'].strip() == "lunch":
            df.at[i, 'weather'] = kilyos_lunch_weather[idx]
        elif row['meal'].strip() == "dinner":
            df.at[i, 'weather'] = kilyos_dinner_weather[idx]
    elif row['campus'].strip() == "Kandilli":
        df.at[i, 'campus'] = 'kandilli'
        if row['meal'].strip() == "breakfast":
            df.at[i, 'weather'] = kandilli_breakfast_weather[idx]
        elif row['meal'].strip() == "lunch":
            df.at[i, 'weather'] = kandilli_lunch_weather[idx]
        elif row['meal'].strip() == "dinner":
            df.at[i, 'weather'] = kandilli_dinner_weather[idx]
    elif row['campus'].strip() == "Anadoluhisari":
        df.at[i, 'campus'] = 'ahisari'
        if row['meal'].strip() == "breakfast":
            df.at[i, 'weather'] = ahisari_breakfast_weather[idx]
        elif row['meal'].strip() == "lunch":
            df.at[i, 'weather'] = ahisari_lunch_weather[idx]
        elif row['meal'].strip() == "dinner":
            df.at[i, 'weather'] = ahisari_dinner_weather[idx]
    elif row['campus'].strip() == "Cevrimdisi":
        df.at[i, 'campus'] = 'cevrimdisi'
        df.at[i, 'weather'] = np.nan

In [118]:
df.to_csv("data/dataset_910.csv", index=False)

### Full dataset

In [119]:
df_1 = pd.read_csv('data/dataset.csv')
df_2 = pd.read_csv('data/dataset_910.csv')

In [120]:
df = pd.concat([df_1, df_2])

In [122]:
df.reset_index(drop=True, inplace=True)

In [123]:
df

,date,day,campus,weather,count,meal,soup,main_dish,vegetarian,side_dishes,dessert
0,2024-01-01,monday,kuzey,14.5800,420,lunch,toyga corba,adana kofte,bugdayli pazi,mercimekli bulgur pilavi,"salata, meyve, portakalli gazoz"
1,2024-01-02,tuesday,kuzey,12.6300,1626,lunch,alaca corba,tas kebabi,zy. bruksel lahanasi,eriste,"profiterol, mor salata, gazoz"
2,2024-01-03,wednesday,kuzey,14.1800,1686,lunch,kafkas corba,meksika soslu tavuk,zy.kereviz,"nohutlu pirinc pilavi, kuskus","salata, ayran, lokma tatlisi"
3,2024-01-04,thursday,kuzey,15.4500,1033,lunch,tavuk suyu corba,kiymali ispanak,bugdayli nohutlu ispanak,"soslu makarna, su boregi","puding, salata, meyve"
4,2024-01-05,friday,kuzey,14.1100,1360,lunch,safak corba,bahcivan kebabi,bezelye yemegi,"sehriyeli pirinc pilavi, bulgur pilavi","cacik, soda, salata"
...,...,...,...,...,...,...,...,...,...,...,...
3496,2024-10-30,wednesday,ahisari,16.8555,281,breakfast,NaN,menemen,yumurta,"yesil zeytin, siyah zeytin, domates, salatalik...","kasar peynir, tereyag, recel, tahin-pekmez"
3497,2024-10-30,wednesday,kuzey,16.8555,181,breakfast,NaN,menemen,yumurta,"yesil zeytin, siyah zeytin, domates, salatalik...","kasar peynir, tereyag, recel, tahin-pekmez"
3498,2024-10-30,wednesday,guney,16.8555,71,breakfast,NaN,menemen,yumurta,"yesil zeytin, siyah zeytin, domates, salatalik...","kasar peynir, tereyag, recel, tahin-pekmez"
3499,2024-10-30,wednesday,kilyos,16.4900,12,breakfast,NaN,menemen,yumurta,"yesil zeytin, siyah zeytin, domates, salatalik...","kasar peynir, tereyag, recel, tahin-pekmez"


In [126]:
df.to_csv("data/dataset_full.csv", index=False)

In [130]:
df[(df['meal']!="breakfast") & (df['campus']!="cevrimdisi")].to_csv("data/dataset_full_refined.csv", index=False)